# Importing Libraries

In [20]:
import numpy as np
import pandas as pd
import config
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from sklearn.preprocessing import MinMaxScaler
from agents.tfagents.dqn import DQNAgent
from agents.tfagents.ppo import PPOAgent
from environments.environment import TradingEnvironment
from environments.wrappers.tf.tfenv import TFRuleTradingEnvironment
from environments.rewards.marketlimitorder import MarketLimitOrderRF
from metrics.trading.pnl import CumulativeLogReturn
from metrics.trading.risk import InvestmentRisk
from metrics.trading.sharpe import SharpeRatio
from metrics.trading.sortino import SortinoRatio
from metrics.trading.drawdown import MaximumDrawdown
from rules.nconsecutive import NConsecutive

# Building Eval Environments

In [21]:
def build_eval_environments(
        dataset_filepath,
        timeframe_size,
        target_horizon_len,
        num_eval_samples,
        fees,
        reward_fn_instance,
        n_consecutive_size: int or None
):
    # Reading dataset
    crypto_dataset_df = pd.read_csv(config.dataset_save_filepath.format(dataset_filepath))
    samples_df = crypto_dataset_df[config.regression_features]

    # Scaling data
    scaler = MinMaxScaler(feature_range=(0, 1.0))
    samples = samples_df.to_numpy(dtype=np.float32)

    num_train_scale_samples = samples.shape[0] - num_eval_samples - target_horizon_len - timeframe_size + 1
    samples[: num_train_scale_samples] = scaler.fit_transform(samples[: num_train_scale_samples])
    samples[num_train_scale_samples: ] = scaler.transform(samples[num_train_scale_samples: ])

    # Constructing timeframes for train, test
    inputs = np.float32([samples[i: i + timeframe_size] for i in range(samples.shape[0] - timeframe_size - target_horizon_len + 1)])

    # Splitting inputs to train-test data
    num_train_inputs = inputs.shape[0] - num_eval_samples
    x_eval = inputs[num_train_inputs:]

    # Computing reward functions for train, test data
    closes = crypto_dataset_df['close'].to_numpy(dtype=np.float32)
    highs = crypto_dataset_df['high'].to_numpy(dtype=np.float32)
    lows = crypto_dataset_df['low'].to_numpy(dtype=np.float32)

    eval_reward_fn = reward_fn_instance(
        timeframe_size=timeframe_size,
        target_horizon_len=target_horizon_len,
        highs=highs[samples.shape[0] - num_eval_samples - timeframe_size - target_horizon_len + 1:],
        lows=lows[samples.shape[0] - num_eval_samples - timeframe_size - target_horizon_len + 1:],
        closes=closes[samples.shape[0] - num_eval_samples - timeframe_size - target_horizon_len + 1:],
        fees_percentage=fees
    )

    assert x_eval.shape[0] == eval_reward_fn.get_reward_fn_shape()[0], \
        f'AssertionError: DimensionMismatch: x_eval: {x_eval.shape}, eval_reward_fn: {eval_reward_fn.get_reward_fn_shape()}'

    eval_env = TFRuleTradingEnvironment(
        env=TradingEnvironment(env_config={
            'states': x_eval,
            'reward_fn': eval_reward_fn,
            'episode_steps': x_eval.shape[0] - 1,
            'metrics': [CumulativeLogReturn(), InvestmentRisk(), SharpeRatio(), SortinoRatio(), MaximumDrawdown()]
        }),
        rules=None if n_consecutive_size is None else [NConsecutive(window_size=n_consecutive_size)]
    )
    tf_eval_env = TFPyEnvironment(environment=eval_env)
    return eval_env, tf_eval_env

# Building & Loading Agents

In [22]:
def build_agent(
        agent_instance,
        env,
        checkpoint_filepath,
        fc_layers,
        conv_layers
):
    agent = agent_instance(
        input_tensor_spec=env.observation_spec(),
        action_spec=env.action_spec(),
        time_step_spec=env.time_step_spec(),
        env_batch_size=env.batch_size,
        checkpoint_filepath=checkpoint_filepath,
        fc_layers=fc_layers,
        conv_layers=conv_layers
    )
    agent.initialize()
    return agent

# Building Eval Method

In [23]:
def eval_tradernet_smurf(tradernet_policy, smurf_policy, tf_env_wrapper):
    time_step = tf_env_wrapper.reset()
    tradernet_policy_state = tradernet_policy.get_initial_state(tf_env_wrapper.batch_size)
    smurf_policy_state = smurf_policy.get_initial_state(tf_env_wrapper.batch_size)
    cumulative_rewards = 0.0
    cumulative_pnls = 0.0
    pnls = []

    while not time_step.is_last():
        smurf_action = smurf_policy.action(time_step=time_step, policy_state=smurf_policy_state).action
        action = 2 if smurf_action == 2 else tradernet_policy.action(time_step=time_step, policy_state=tradernet_policy_state).action
        time_step = tf_env_wrapper.step(action)
        reward = time_step.reward.numpy()[0]
        cumulative_rewards += reward

        if action != 2:
            cumulative_pnls += reward
        pnls.append(cumulative_pnls)
    return cumulative_rewards, pnls

# Building Configs

In [24]:
datasets_dict = {'BTC': 'BTC', 'ETH': 'ETH', 'ADA': 'ADA', 'XRP': 'XRP', 'LTC': 'LTC'}
agent_dict = {
    'PPO': {
        'agent_instance': PPOAgent,
        'fc_layers': [256, 256],
        'conv_layers': [(32, 3, 1)]
    },
    'DDQN': {
        'agent_instance': DQNAgent,
        'fc_layers': [256, 256],
        'conv_layers': [(32, 3, 1)]
    },
}
env_dict = {
    'timeframe_size': 12,
    'target_horizon_len': 20,
    'num_eval_samples': 2250,
    'fees': 0.007,
    'n_consecutive_size': 2
}

reward_fn_name = 'Market-Limit Orders'

# Running Experiments for TraderNet + Smurf

In [25]:
for agent_name, agent_config in agent_dict.items():
    for dataset_name, dataset_filepath in datasets_dict.items():
        eval_env, tf_eval_env = build_eval_environments(
            dataset_filepath=dataset_filepath,
            reward_fn_instance=MarketLimitOrderRF,
            **env_dict
        )
        tradernet = build_agent(
            env=tf_eval_env,
            checkpoint_filepath=f'database/storage/checkpoints/experiments/tradernet/{agent_name}/{dataset_name}/{reward_fn_name}/',
            **agent_config
        )

        smurf_agent = build_agent(
            env=tf_eval_env,
            checkpoint_filepath=f'database/storage/checkpoints/experiments/smurf/DDQN/{dataset_name}/{reward_fn_name}/',
            **agent_config
        )
        average_returns, pnls = eval_tradernet_smurf(
            tradernet_policy=tradernet.policy,
            smurf_policy=smurf_agent.policy,
            tf_env_wrapper=tf_eval_env
        )
        metrics = {
            'average_returns': [average_returns],
            **{key: [metric] for key, metric in eval_env.get_episode_metrics().items()}
        }
        results_df = pd.DataFrame(metrics)
        results_df.to_csv(f'experiments/integrated/{agent_name}/{dataset_name}_{reward_fn_name}_metrics.csv', index=False)

        print(results_df, '\n')

        episode_pnls_df = pd.DataFrame(pnls)
        episode_pnls_df.to_csv(f'experiments/integrated/{agent_name}/{dataset_name}_{reward_fn_name}_eval_cumul_pnls.csv', index=False)

        print(episode_pnls_df.tail(5))

   average_returns  Cumulative Log Returns  Investment Risk    Sharpe  \
0         4.521696                11.23179         0.470145  1.328915   

   Sortino  Maximum Drawdown  
0   3.5772          0.817935   

             0
2244  7.962402
2245  7.951263
2246  7.945543
2247  7.952276
2248  7.965678
   average_returns  Cumulative Log Returns  Investment Risk    Sharpe  \
0        31.467259               34.562676         0.331935  1.750114   

     Sortino  Maximum Drawdown  
0  46.450316          0.126021   

              0
2244  31.441576
2245  31.435936
2246  31.430761
2247  31.445079
2248  31.467259


   average_returns  Cumulative Log Returns  Investment Risk    Sharpe  \
0        10.552246               24.101656         0.336657  1.610798   

     Sortino  Maximum Drawdown  
0  13.433975          5.565994   

              0
2244  11.159865
2245  11.178231
2246  11.194291
2247  11.219376
2248  11.181086


C:\Users\kochlis\Documents\Research\TraderNetv2\metrics\trading\sharpe.py:20: RuntimeWarning: invalid value encountered in double_scalars
  return np.exp(average_returns/std_returns)
C:\Users\kochlis\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\kochlis\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\kochlis\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


   average_returns  Cumulative Log Returns  Investment Risk  Sharpe  Sortino  \
0       -70.379562                     0.0                0     NaN      NaN   

   Maximum Drawdown  
0                 0   

        0
2244  0.0
2245  0.0
2246  0.0
2247  0.0
2248  0.0
   average_returns  Cumulative Log Returns  Investment Risk    Sharpe  \
0         3.136148               19.854592         0.361042  1.537121   

    Sortino  Maximum Drawdown  
0  9.944106          0.173675   

             0
2244  8.513062
2245  8.535203
2246  8.566891
2247  8.598818
2248  8.544265
   average_returns  Cumulative Log Returns  Investment Risk    Sharpe  \
0       -27.092092                2.560518         0.388186  1.159418   

    Sortino  Maximum Drawdown  
0  1.317127          1.088456   

             0
2244 -0.231092
2245 -0.231092
2246 -0.231092
2247 -0.231092
2248 -0.231092
   average_returns  Cumulative Log Returns  Investment Risk    Sharpe  \
0         8.755367               27.216975         0.2